In [11]:
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install tqdm
from tqdm import tqdm
import requests
import os
from dotenv import load_dotenv
import json

In [12]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [13]:
df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
0,12834217_1,572066,1346,0,False,"As of March 13th , 2014 , the booklet had been...",False,False
1,12834217_2,572066,1346,0,False,In order to help increase the booklets downloa...,True,False
2,12834217_3,572066,1346,0,False,( Simply copy and paste the following text int...,False,False
3,12834217_4,572066,1346,0,True,Click below for a FREE download of a colorfull...,True,False
4,12834217_5,572066,1346,0,False,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,False,False
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,False,"Billy - `` That guy would n't leave me alone ,...",False,False
10940,33677019_1,735154,1388,0,False,Wish we at least had a Marine Le Pen to vote f...,False,False
10941,33677019_2,735154,1388,0,False,Its like the choices are white genocide candid...,False,False
10942,33677053_1,572266,1388,0,True,Why White people used to say that sex was a si...,True,False


In [14]:
file_names = ['prompt-variations/v0.txt', 'prompt-variations/v1.txt']
SYSTEM_PROMPT = []
for file_name in file_names:
    with open(file_name, 'r') as file:
        SYSTEM_PROMPT.append(file.read())

In [15]:
load_dotenv()
PERPLEXITY_API_KEY = os.getenv("PPLX_KEY")
VALID_MODELS = ['codellama-34b-instruct', 'llama-2-70b-chat', 'mistral-7b-instruct', 'mixtral-8x7b-instruct', 'pplx-7b-chat', 'pplx-70b-chat']

def run_model(prompt_id, model, sample_size, print_results=False):
    if model not in VALID_MODELS:
        raise ValueError(f'Invalid model {model}. Valid options are {VALID_MODELS}')

    sample_df = testing_df.sample(n=sample_size)

    results = []
    for index, row in tqdm(sample_df.iterrows(), total=sample_df["text"].count(), unit="generations", desc=f'Running {model} (Promt {prompt_id})'):
        payload = {
            "model": model,
            "messages": [
                {
                    "role": "system",
                    "content": SYSTEM_PROMPT[prompt_id]
                },
                {
                    "role": "user",
                    "content": row["text"]
                }
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization": f"Bearer {PERPLEXITY_API_KEY}"
        }

        response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)
        if response.status_code != 200:
            print(f"Error")
        data = json.loads(response.text)
        answer = data['choices'][0]['message']['content']
        results.append({
            'prompt_id': prompt_id,
            'model': model,
            'sample_size': sample_size,
            "text": row["text"],
            "answer": answer,
            "labeled_hateful": row["hate"]
        })
        if print_results:
            print(row["text"])
            if lines := [line for line in answer.split("\n") if "hate_speech_probability" in line]:
                print(lines)
            else:
                print(answer)

    return pd.DataFrame(results)

In [17]:
import time
time.sleep(60*20)

In [18]:
total_samples = len(testing_df)
all_runs = pd.read_pickle("all_runs.pkl")

CONFIG = [
    #(0, 'mistral-7b-instruct', total_samples),
    (0, 'mixtral-8x7b-instruct', total_samples),
    (0, 'pplx-70b-chat', total_samples),
    (0, 'pplx-7b-chat', total_samples),
    (1, 'mistral-7b-instruct', total_samples),
    (1, 'mixtral-8x7b-instruct', total_samples),
]

for config in CONFIG:
    run = run_model(*config)
    all_runs = pd.concat([all_runs, run])
    pd.DataFrame(all_runs).to_pickle("all_runs.pkl")

  9%|▉         | 43/478 [02:31<25:35,  3.53s/it]


KeyError: 'choices'

In [ ]:
all_runs

In [19]:

all_runs = pd.read_pickle("all_runs.pkl")
all_runs = all_runs[all_runs["model"]=="mistral-7b-instruct"]
failed_runs = all_runs[all_runs["model"]!="mistral-7b-instruct"]